<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-UT/blob/main/DataAlgo_UT(012)_EightQueens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.2.2. エイトクイーン (Eight queens)

バックトラックの別の例として，エイトクィーン問題を取り上げる．


**いつもの約束**  
１つのコードセルだけの実行は Ctrl + Enter．  
エディタで「インデント幅（スペース）は4で表示」「行番号を表示」「インデントガイドを表示」．  
内部では日本語はUTF-8で表現されている．


# 準備

インスタンスに接続し起動する．  
下記の手順でGoogle Driveをマウントする．  
マウント先に移動し，作業フォルダとする．  
これによって，インスタンスがリセットされてもGoogle Drive内にファイルが保存されるようにする．

In [ ]:
!echo "Google Driveをマウントします"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "今回の作業用フォルダを作成しそこに移動します"
%cd /content/drive/My\ Drive/
%mkdir -p 202104_DataAlgo/DA2021_012
%cd       202104_DataAlgo/DA2021_012
!ls
!echo "日本時間表示"
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime
!date

# エイトクィーン

**問題**

チェスの女王(queen)は縦横斜めに好きなだけ移動できる．将棋で言えば飛車と角の両方を兼ね備える存在で，最も強力な駒とされる．エイトクィーン問題とは，女王駒を8つ用意し，

1. チェスの8x8の盤の中で
2. お互いに取られない配置に
3. 8女王を配置する

ことができるかどうかを調べる問題である．

より一般化して，NxNの盤を考え，N-Queens問題とすることも考えられる．




# 全解探索法

盤上には駒を置けるマスが8<sup>2</sup>=64ある．そこに同じ8個の駒を置くので，組み合わせの数は <sub>64</sub>C<sub>8</sub>ということになる．これはすなわち解候補である．

<sub>64</sub>C<sub>8</sub> = (64 x ... x 57) / (8 x ... x 1) = 4,426,165,368

この解候補それぞれについて，「お互いに取られない位置に」あることを確認すればよい．


# バックトラック法

8つの女王を配置していく途中で，「チェスの8x8の盤の中で」「お互いに取られない配置」を満たせなくなった時点で，その先に解はない．

女王は4直線方向に移動できる．これを，縦移動とそれ以外に分けて考える．

縦移動によって取られることを防ぐために，8つの女王を各列に1つずつ置く．各列には8マスあるから，考え得る解候補は8<sup>8</sup>=16,777,216にまで減ることになる．

（横方向によって取られることを防ぐことまで考えると，考え得る解候補はさらに減らすことができる）

今，左端の列から配置を決定していくことにする．次の列での女王を，過去に配置した女王のどれからも取られない位置に置けばよい．どこに置いても取られてしまう場合はバックトラック（後戻り）することになる．



# バックトラック法によるCプログラム

**目標**

NxNのチェスの盤におけるN-Queens問題を解くCプログラムを作成する．

**説明**

横方向と斜め2方向から取られることを防ぐため，それぞれその方向がすでにこれまでの女王によって占拠されていると考える．３方向で取られない位置にあることを確認するために，YY, RD, LDという三つの配列を用いる．注目するマスに対応するこれらの配列要素がマーク済みであれば，取られることになるのでそのマスにクイーンを置くことはできない．（下図参照）

バックトラック時に記録の削除（原状復帰）が行われるべきであるが，board[x]については元の値に意味がないので，原状復帰に相当する部分が無い．


**コード**

横方向の占拠を表すのに必要な配列の大きさはNであるが，斜め2方向の占拠を表すのに必要な配列の大きさはそれぞれNではなく2N-1である．

**備考**




![eightqeens](https://user-images.githubusercontent.com/45651568/90338504-0fb2a880-e025-11ea-81b0-0a96c52e8b77.png)


In [ ]:
%%writefile NQueens_J.c
// N Queens, back track version
//    kameda[ccs]tsukuba.ac.jp, 2020.
#include <stdio.h>

int n_answer = 0;

#define N 8
int rd[2*N-1]; // right-down direction checker
int ld[2*N-1]; // left-down  direction checker
int yy[N];     // Y-line checker
int board[N]; // if a queen is at (X,Y), board[X]=Y

// Counting the number of answers and display its layout
void print_board(void){
	int x, y;
	printf("Answer No. %d ==========\n", n_answer++);
	for (y = 0; y < N; y++) {
		for (x = 0; x < N; x++) 
			printf("%c", board[y]==x ? 'Q' : '.');
		printf("\n");
	}
}

// n queens are examined from X = 0 to n-1), now try to place a queen on X=x
void placeaqueen(int x){
	int y;

	// Rearch to an answer
	if (x == N) {
		print_board();
		return;
	}

	// check all y's at X=x
	for (y = 0; y < N; y++) {
		if (yy[y] == 0 && rd[x+y] == 0 && ld[x-y+N-1] == 0) {
			yy[y] =  1 ;  rd[x+y] =  1 ;  ld[x-y+N-1] =  1; // Mark / yy, rd, ld
			board[x] = y;                                   // Mark / board
			placeaqueen(x+1);
			yy[y] =  0 ;  rd[x+y] =  0 ;  ld[x-y+N-1] =  0; // Unmark / yy, rd, ld
			                                                // Unmark / board (actually nothing to do)
		}
	}
}

// Main function
int main(int argc, char *argv[]){
	int x, d;

	// No queens there at the begining
	for (x = 0; x < N; x++) {
		yy[x] = 0;
	}
	for (d = 0; d < 2*N-1; d++) {
		rd[d] = 0; 
		ld[d] = 0; 
	}

	placeaqueen(0); // Starting with X = 0

	return 0;
}



コンパイルしてエラーが無いことを確認．

In [ ]:
!gcc -Wall -o NQueens_J NQueens_J.c

実行．

In [ ]:
!time ./NQueens_J

# 節末課題

1. NQueens_Jの計算量  
NQueens_J プログラムの時間計算量と空間計算量を議論せよ．盤は正方とし，その一辺のマスの数をNとする．

2. 斜め方向の衝突判定  
列xの行yに女王があるとき，右下がり方向に取り合いになる（衝突し得る）女王はすべて x+y が同じになり，しかもその値が 0 から　2N-2 の間に収まることを示せ．同じく，右上がり方向に取り合いになる（衝突し得る）女王はすべて x-y+N-1 が同じで，かつその値は 0 から 2N-2 までの間に収まることを示せ．なお，ここで右下がりとは，X軸正を右に，Y軸正を上に取った場合である．

3. 解候補の必要条件の実装  
NQueens_J プログラムにおいて，「チェスの8x8の盤の中で」「お互いに取られない配置に」「8女王を配置する」ことを確認している実装部分をそれぞれ示せ．






# 出典

筑波大学工学システム学類  
データ構造とアルゴリズム  
担当：亀田能成  
2021/06/09 初版